In [ ]:
'''
Computes the average target function output for the uniform baseline.
'''


from pyoptes.optimization.budget_allocation import target_function as f
from pyoptes import create_graph, compute_average_otf_and_stderr
from pyoptes import create_test_strategy_prior, map_low_dim_x_to_high_dim

import argparse
import numpy as np
from tqdm import tqdm
from scipy.stats.mstats import mjci


def rms_tia(n_infected_animals):
    values = n_infected_animals**2
    estimate = np.sqrt(np.mean(values, axis=0))
    stderr = np.std(values, ddof=1, axis=0) / np.sqrt(values.shape[0])
    stderr = stderr/(2*estimate)
    return estimate, stderr

In [ ]:
scale_total_budget = 1
n_nodes = 120
sentinels = 12
graph_type = 'syn'
n_runs = 2
n_simulations = 2
path_networks = '../../networks/data'
statistic = rms_tia
n = 0
total_budget = scale_total_budget * n_nodes

In [ ]:
transmissions, capacities, degrees = create_graph(n, graph_type, n_nodes, path_networks)

f.prepare(n_nodes=n_nodes,
          capacity_distribution=capacities,
          pre_transmissions=transmissions,
          p_infection_by_transmission=0.5,
          delta_t_symptoms=60,
          expected_time_of_first_infection=30,
          static_network=None,
          use_real_data=False)

# create a list of test strategies based on different heuristics
prior, prior_node_indices, prior_parameter = \
    create_test_strategy_prior(n_nodes=n_nodes,
                               node_degrees=degrees,
                               node_capacities=capacities,
                               total_budget=total_budget,
                               sentinels=sentinels,
                               mixed_strategies=True,
                               only_baseline=False)

# evaluate the strategies in the prior
list_prior_tf = []
list_prior_stderr = []

for i, p in tqdm(enumerate(prior), leave=False, total=len(prior)):

    p = map_low_dim_x_to_high_dim(x=p,
                                  number_of_nodes=n_nodes,
                                  node_indices=prior_node_indices[i])

    m, stderr = f.evaluate(budget_allocation=p,
                           n_simulations=n_simulations,
                           parallel=True,
                           num_cpu_cores=-1,
                           statistic=statistic)
    list_prior_tf.append(m)
    list_prior_stderr.append(stderr)

In [ ]:
np.shape(prior)

In [ ]:
np.shape(list_prior_tf)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from neural_process import NeuralProcess

x_dim = 12
y_dim = 1
r_dim = 50  # Dimension of representation of context points
z_dim = 50  # Dimension of sampled latent variable
h_dim = 50  # Dimension of hidden layers in encoder and decoder

neuralprocess = NeuralProcess(x_dim, y_dim, r_dim, z_dim, h_dim)

In [ ]:
# Create a set of 100 target points, with shape 
# (batch_size, num_points, x_dim), which in this case is
# (1, 100, 1)
x_target = torch.Tensor(np.ones((100,12)))
# x_target = [x_target, x_target]
print(np.shape(x_target))
x_target = x_target.unsqueeze(0)
print(np.shape(x_target))
for i in range(64):
    z_sample = torch.randn((1, z_dim))  # Shape (batch_size, z_dim)
    # Map x_target and z to p_y_target (which is parameterized by a 
    # normal with mean mu and std dev sigma)
    mu, _ = neuralprocess.xz_to_y(x_target, z_sample)
    # Plot predicted mean at each target point (note we could also
    # sample from distribution but plot mean for simplicity)
#     print(np.shape(mu))

In [ ]:
from torch.utils.data import DataLoader, Dataset

class testdataset(Dataset):
    def __init__(self,x,y):
        self.x = x
        self.y = y
        print(np.shape(self.x))
        print(np.shape(self.y))
    def __getitem__(self,index):
        return self.x[index],self.y[index]
    def __len__(self):
        return len(self.x)

In [ ]:
dataset = testdataset(prior, list_prior_tf)

In [ ]:
from training import NeuralProcessTrainer

batch_size = 2
num_context = 4
num_target = 4

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
optimizer = torch.optim.Adam(neuralprocess.parameters(), lr=3e-4)
np_trainer = NeuralProcessTrainer(device, neuralprocess, optimizer,
                                  num_context_range=(num_context, num_context),
                                  num_extra_target_range=(num_target, num_target), 
                                  print_freq=200)

neuralprocess.training = True
np_trainer.train(data_loader, 30)